# 第8章 对象引用、可变性和垃圾回收

## 8.1 变量不是盒子

In [1]:
a = [1, 2, 3]
b = a
b.append(4)
b

[1, 2, 3, 4]

In [2]:
a

[1, 2, 3, 4]

## 8.2 标识、相等性和别名

In [3]:
charles = {"name": "Charles L.Dodgson", "born": 1832}
lewis = charles

In [4]:
lewis is charles

True

### 8.2.1 在==和is之间选择
==运算符比较两个对象的值(对象中保存的数据), 而is比较对象标志

然而, 在变量和单例值之间比较时, 应该使用is

In [5]:
x = None
x is None

True

is运算符比==快, 因为它不能重载, 它会直接比较两个对象的id

### 8.2.2 元组的相对不可变性

In [6]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [7]:
id(t1[-1])

140112034544328

In [8]:
t1[-1].append(99)

In [9]:
t1

(1, 2, [30, 40, 99])

## 8.3 默认浅复制

In [10]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)

In [11]:
l1 == l2

True

In [12]:
l1 is l2

False

In [13]:
l1  = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print("l1:", l1)
print("l2:", l2)
l2[1] += [33, 22]
l2[2] += (10, 11)
print("l1:", l1)
print("l2:", l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


为任意对象做深复制和浅复制

copy模块的copy()和deepcopy()用法能为任意对象做浅深拷贝

In [14]:
class Bus(object):
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [15]:
import copy
bus1 = Bus(["Alice", "Bill", "Claire", "David"])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

In [16]:
id(bus1), id(bus2), id(bus3)

(140112034526208, 140112034526824, 140112034526432)

In [17]:
bus1.drop("Bill")

In [18]:
bus2.passengers

['Alice', 'Claire', 'David']

In [19]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [20]:
# deepcopy的循环拷贝
a = [10, 20]
b = [a, 30]

In [21]:
a.append(b)

In [22]:
a

[10, 20, [[...], 30]]

In [23]:
c = copy.deepcopy(a)

In [24]:
c

[10, 20, [[...], 30]]

In [25]:
len(c)

3

## 8.4 函数的参数作为引用时
Python唯一支持的参数传递模式是共享传参。

共享传参指函数的各个形式参数获得实参中各个引用的副本

In [26]:
def f(a, b):
    a += b
    return a

In [27]:
x = 1
y = 2
f(x, y)

3

In [28]:
x,y

(1, 2)

In [29]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [30]:
a,b

([1, 2, 3, 4], [3, 4])

In [31]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [32]:
t, u

((10, 20), (30, 40))

### 8.4.1 不要使用可变类型作为参数的默认值

In [33]:
class HauntedBus:
    
    def __init__(self, passengers = []):  # 产生问题的根源就在于默认值是在定义函数的时候进行计算的
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [34]:
b1 = HauntedBus(["Alice", "Bill"])
b1.passengers

['Alice', 'Bill']

In [35]:
b1.pick("Charlie")

In [36]:
b1.passengers

['Alice', 'Bill', 'Charlie']

In [37]:
bus2 = HauntedBus()

In [38]:
bus2.pick("Carrie")
bus2.passengers

['Carrie']

In [39]:
bus3 = HauntedBus()  # 如果不传入乘客就会使用共享的默认列表
bus3.passengers

['Carrie']

In [40]:
bus3.pick("Dave")

In [41]:
bus2.passengers

['Carrie', 'Dave']

In [42]:
bus2.passengers is bus3.passengers

True

## 8.5 del和垃圾回收
del语句删除名称, 而不是对象本身。它能删除是因为没有引用指向对象的时候, 对象会被垃圾回收。

In [43]:
import weakref
s1 = {1, 2, 3}
s2 = s1

In [44]:
def bye():
    print("Gone with the wind...")

In [45]:
ender = weakref.finalize(s1, bye)  # 在s1注册bye的回调函数

In [46]:
ender.alive

True

In [47]:
del s1

In [48]:
ender.alive

True

In [49]:
s2 = "spam"

Gone with the wind...


In [50]:
ender.alive

False

## 8.6 弱引用
正是因为有引用，对象才会在内存中存在。当对象的引用数量归0的时候。垃圾回收程序会把对象销毁。但是，有时需要引用对象，而不让对象存在的时间超过

所需的时间，这经常用在缓存中。

弱引用不会增加对象的引用数目。

In [85]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)

In [86]:
wref

<weakref at 0x7f6e527d7098; to 'set' at 0x7f6e60115ba8>

In [87]:
wref()

{0, 1}

In [88]:
a_set = {1,2,3}

In [89]:
wref() is None

False

In [90]:
wref() is None  # 这里在python控制台中应该是True, 但是应该是jupyter有什么引用的机制

False

### 8.6.1 WeakValueDictionary简介
WeakValueDictionary类实现的是一种可变映射, 里面的值是对象的弱引用。被引用的对象在程序其他地方被删除时，对于的键会从WeakValueDictionary中

删除。因此，WeakValueDictionary经常用于缓存。

In [94]:
class Cheese(object):
    
    def __init__(self, kind):
        self.kind = kind
        
    def __repr__(self):
        return "Cheese({})".format(self.kind)

In [96]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese("Red Leicester"), Cheese("Tilsit"), Cheese("Brie"), Cheese("Parmesan")]

In [97]:
for cheese in catalog:
    stock[cheese.kind] = cheese

In [98]:
sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [99]:
del catalog

In [101]:
sorted(stock.keys())  # 这里删除之后应该会只剩下Parmesan, jupyter的引用机制

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [102]:
del cheese

In [103]:
sorted(stock.keys())  # 现在应该什么都不剩

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

### 8.6.2 弱引用的局限
不是每个Python对象都可以作为弱引用的目标。基本的list和dict实例不能作为所指对象, 但是可以引用它们的子类。

int和tuple连子类都不能作为弱引用的对象。

In [104]:
class MyList(list):
    """list子类, 实例可以作为弱引用的目标"""

## 8.7 Python对不可变类型施加的把戏
这是针对不同编译器优化的结果, 下面是Cpython的。

In [105]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t1 is t2

True

In [106]:
s1 = "ABC"
s2 = "ABC"
s1 is s2  # 共享字符串

True

In [107]:
t1 = (1, 2, 3)
t2 = (1, 2, 3)
t1 is t2

False